# Continuation Power Flow

Demonstrates continuation power flow (CPF) analysis for voltage stability
assessment. The notebook sets up interface transfers, runs the CPF solver
to trace the PV curve, and visualizes the nose point that marks the voltage
stability limit.

In [ ]:
import numpy as np
from esapp import PowerWorld
from esapp.components import Bus, Gen, Area

In [ ]:
# This cell is hidden in the documentation.
import ast

with open('../data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

pw = PowerWorld(case_path)

In [ ]:
# Plotting functions (hidden from documentation)
import sys; sys.path.insert(0, "..")
from plot_helpers import plot_pv_curve

## 1. Define Interface Transfer

The continuation power flow traces the PV curve by increasing a transfer
pattern (the `interface` vector) and solving power flow at each step. It uses
a predictor-corrector method with tangent vectors from the augmented Jacobian
for accurate nose-point tracking.

In [ ]:
# Solve base case
V_base = pw.pflow()
print(f"Base case min voltage: {np.abs(V_base).min():.4f} pu")

# Critical bus: lowest voltage magnitude
critical_bus_idx = np.argmin(np.abs(V_base))
print(f"Critical bus index: {critical_bus_idx}")
print(f"Critical bus voltage: {np.abs(V_base[critical_bus_idx]):.4f} pu")

# Build interface vector: uniform load increase at all buses
n_buses = len(pw[Bus])
interface = np.ones(n_buses)
interface /= np.sum(interface)  # normalize to 1 MW total

## 2. PV Curve

The PV curve shows how voltage at a critical bus varies with increasing
power transfer. The nose point indicates the maximum transfer before
voltage collapse.

In [ ]:
# Collect PV curve data points
mw_points = []
v_points = []

for mw in pw.statics.continuation_pf(
    interface=interface,
    initialmw=0,
    step_size=0.05,
    min_step=0.001,
    max_step=0.1,
    maxiter=200,
    verbose=True,
    restore_when_done=True,
):
    V = pw.statics.voltage()
    v_critical = np.abs(V[critical_bus_idx])
    mw_points.append(mw)
    v_points.append(v_critical)

print(f"\nCollected {len(mw_points)} points")
if mw_points:
    print(f"Transfer range: {min(mw_points):.1f} to {max(mw_points):.1f} MW")

In [ ]:
plot_pv_curve(mw_points, v_points)